In [43]:
from qiskit import *
from qiskit_aer import QasmSimulator
from qiskit.visualization import plot_histogram, plot_error_map


In [44]:
from qiskit_ibm_runtime import QiskitRuntimeService

In [45]:
service = QiskitRuntimeService()

In [46]:
def XOR(chunk1, chunk2):
    
    s = ""
    for char1, char2 in zip(chunk1, chunk2):
        xo = ord(char1)^ord(char2)
        s += str(chr(xo))
    return s

In [47]:
def Rotate(chunk1, j):
    shift = 1
    s = ""
    size = len(chunk1)
    for i in range(shift, size):
        s += str(chunk1[i])
    
    for j in range(0,shift):
        s += str(chunk1[j])
    return s

In [48]:
def KEY(encrypt):
    numQ = 8; 
    circ = QuantumCircuit(numQ, numQ)
    circ.h(range(numQ))
    circ.measure(range(numQ), range(numQ))
    sim = QasmSimulator()
    comp = transpile(circuits=circ, backend=sim)
    job = sim.run(comp, shots=3000)
    result = job.result()
    counts = result.get_counts(circ)
    
    def QRNG():
        
        jobR = sim.run(transpile(circ, sim), shots=1000)
        result = jobR.result()
        count = result.get_counts(circ)
        key = ""
        for val in list(count.keys()):
            key += str(chr(int(val,2)))
            
        return key  
    
    def AES_Expan(message, key):
        keyc = list(key)
        rounds = []
        encry = ""
        size = len(keyc)
        
        for i in range(0, size, 10):
            if(i+3 > size):
                rounds.append(keyc[i:size])
            else:
                rounds.append(keyc[i:i+10])            
        size = len(rounds)*10
        chec = 0
        for j in range(5,size):
            if(chec >= len(message)):break
            if(j%5 == 0):
                encry += XOR(XOR(rounds[j-1], rounds[j-5]), message[chec])
                rounds.append(XOR(rounds[j-1], rounds[j-5]))
            else:
                encry += XOR(XOR(Rotate(rounds[j-2], j), "#@^$#$^"), message[chec])
                rounds.append(XOR(Rotate(rounds[j-2], j), "#@^$#$^"))
            chec += 1
            
        return encry
    
    return AES_Expan(encrypt,QRNG())


In [49]:
KEY("The old key lay in Mira’s palm, warm despite the cold night. She had found it in her grandmother’s sewing box, tied with a red ribbon.The note read: For when you’re ready.Drawn by instinct, she walked to the garden shed she’d never entered. The lock clicked open. Inside was a trunk filled with photographs—people she’d never met, all smiling at her as if they knew her.At the bottom lay a letter addressed to My future, brave girl.Mira smiled through tears. She was ready.")

'\x12XÚ¾n©¬%{\x99Ã¸g0\x1b\x8aÅ\x95oìÊD\x0f⃛\x15g-5Ï{\x154ÉU\x08µÎà(°{ù\x9a~{±ÈuiÅ>^\x07V/*ý\x1bõz;g_È\x9b8`\t¼\x17\x0cTo\x18\x93yiúÊÍ7À\x01\x8cºÇàt3 L\x00\x1bø\x0b¢\u20f6Lm\x00Ö\x188\x05Y\xadäºÎ\x88Ä(0yÓo³Ê\x04(\x83ÝSã\x1eé\x86\x86ª\x89K-Äq\x8f;e\x9eÞË{æ\x1c=×¹jÌl\x9d\taµJ¿\x97²©Äg\x82₲l¸¨ÏApØä-±´òòß;\x08¤)\x939r/ß\x11\x0fm-wS\x96¢1\x1aç5\x95È_\x98«qFE$3n\x8fµZ_¡\x16\x05>\x95Øè\x83Î\xa0g₀Z©6\x0eKßêÕ3ï»ho\x7fÕäÎ\x15>:y'